In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
using System.IO;

var connStr = File.ReadAllText(@"C:\Users\Pedro_Carvalho1\conn_str.txt");

#!share --from csharp connStr

In [ ]:
#!connect mssql --kernel-name sqlstudy --connection-string @connStr

In [7]:
#!sql-sqlstudy
IF OBJECT_ID('dbo.Employee', 'U') IS NOT NULL
    DROP TABLE dbo.Employee;
GO

CREATE TABLE dbo.Employee (
    id INT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    department VARCHAR(50) NOT NULL,
    managerId INT NULL
);


Commands completed successfully.

Commands completed successfully.

In [9]:
#!sql-sqlstudy
INSERT INTO dbo.Employee (id, name, department, managerId) VALUES
(101, 'John',  'A', NULL),
(102, 'Dan',   'A', 101),
(103, 'James', 'A', 101),
(104, 'Amy',   'A', 101),
(105, 'Anne',  'A', 101),
(106, 'Ron',   'B', 101);

(6 rows affected)

##### Exercício

In [ ]:
570. Managers with at Least 5 Direct Reports

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
| department  | varchar |
| managerId   | int     |
+-------------+---------+

id is the primary key (column with unique values) for this table.
Each row of this table indicates the name of an employee, their department, and the id of their manager.
If managerId is null, then the employee does not have a manager.
No employee will be the manager of themself.


>> QUESTION:
Write a solution to find managers with at least five direct reports.
Return the result table in any order.
The result format is in the following example.

Example 1:

Input: 
Employee table:
+-----+-------+------------+-----------+
| id  | name  | department | managerId |
+-----+-------+------------+-----------+
| 101 | John  | A          | null      |
| 102 | Dan   | A          | 101       |
| 103 | James | A          | 101       |
| 104 | Amy   | A          | 101       |
| 105 | Anne  | A          | 101       |
| 106 | Ron   | B          | 101       |
+-----+-------+------------+-----------+

Output: 
+------+
| name |
+------+
| John |
+------+


In [ ]:
#!sql-sqlstudy
-- Ver todos os registros
SELECT * FROM dbo.Employee;

-- Ver subordinados de John (id 101)
SELECT * FROM dbo.Employee WHERE managerId = 101;

-- exercicio::

-- Write a solution to find managers with at least five direct reports.
-- Return the result table in any order.
-- The result format is in the following example.

-- VISAO GERAL DO INNER JOIN
SELECT 
    MANAG.ID AS MANAGER_ID, MANAG.NAME AS MANGER_NAME, MANAG.managerId AS MANAGER_MANAGER_ID,
    EMP.ID, EMP.NAME, EMP.managerId
FROM dbo.Employee MANAG
INNER JOIN dbo.Employee EMP ON MANAG.id = EMP.managerId

-- RESOLVENDO COM AGREGACAO SIMPLES
SELECT
    MANAG.NAME AS MANAGE_NAME,
    COUNT(*) AS #MANAGES
FROM dbo.Employee MANAG
INNER JOIN dbo.Employee EMP ON MANAG.id = EMP.managerId
GROUP BY MANAG.ID, MANAG.NAME
HAVING COUNT(*) > 4


(6 rows affected)

(5 rows affected)

(5 rows affected)

(1 row affected)

id,name,department,managerId
101,John,A,<null>
102,Dan,A,101
103,James,A,101
104,Amy,A,101
105,Anne,A,101
106,Ron,B,101


id,name,department,managerId
102,Dan,A,101
103,James,A,101
104,Amy,A,101
105,Anne,A,101
106,Ron,B,101


MANAGER_ID,MANGER_NAME,MANAGER_MANAGER_ID,ID,NAME,managerId
101,John,<null>,102,Dan,101
101,John,<null>,103,James,101
101,John,<null>,104,Amy,101
101,John,<null>,105,Anne,101
101,John,<null>,106,Ron,101


MANAGE_NAME,#MANAGES
John,5


In [ ]:
#!sql-sqlstudy
-- VISAO GERAL DO INNER JOIN
SELECT 
    MANAG.ID AS MANAGER_ID, MANAG.NAME AS MANGER_NAME, MANAG.managerId AS MANAGER_MANAGER_ID,
    EMP.ID, EMP.NAME, EMP.managerId
FROM dbo.Employee MANAG
INNER JOIN dbo.Employee EMP ON MANAG.id = EMP.managerId

-- VISAO GERAL, COM LEFT JOIN
SELECT 
    MANAG.ID AS MANAGER_ID, MANAG.NAME AS MANGER_NAME, MANAG.managerId AS MANAGER_MANAGER_ID,
    EMP.ID, EMP.NAME, EMP.managerId
FROM dbo.Employee MANAG
left JOIN dbo.Employee EMP ON MANAG.id = EMP.managerId

-- EXPLICANDO::
-- PERCEBA QUE NO > LEFT < JOIN A TABELA QUE CHAMAMOS DE "dbo.Employee MANAG", QUE EH A TABELA QUE COMPOE AS 3 PRIMEIRAS COLUNAS
-- DA ESQUERDA PARA A DIREITA, MOSTRA VARIAS LINHAS AONDE AS COLUNAS 4 , 5 E 6 ESTAO COMO NULL
-- ISSO PORQUE NAO EXISTE NENHUMA COMBINACAO POSSIVEL DE ACORDO COM A REGRA DO LEFT JOIN, QUE EH 
-- "left JOIN dbo.Employee EMP ON MANAG.id = EMP.managerId", OU SEJA, PEGA A MINHA TABELA INICIAL DA ESQUERDA, dbo.Employee MANAG, 
-- E LEFT JOIN NA TABELA DA DIREITA, dbo.Employee EMP, ONDE MANAG.id = EMP.managerId
-- LOGO, NAO EXISTEM, NA TABELA DA DIREITA, NENHUM RESULTADO ONDE MANAG.id SEJA IGUAL A EMP.managerId
-- POR ISSO AS COLUNAS 4, 5 E 6 FICAM EM NULL

-- E TAMBEM EH POR ISSO QUE, SE USARMOS O INNER JOIN ESSAS LINHAS DESAPARECEM
-- PORQUE NO INNER JOIN RETORNA APENAS LINHAS QUE DEEM MATCH EM AMBAS TABELAS



(5 rows affected)

(10 rows affected)

MANAGER_ID,MANGER_NAME,MANAGER_MANAGER_ID,ID,NAME,managerId
101,John,<null>,102,Dan,101
101,John,<null>,103,James,101
101,John,<null>,104,Amy,101
101,John,<null>,105,Anne,101
101,John,<null>,106,Ron,101


MANAGER_ID,MANGER_NAME,MANAGER_MANAGER_ID,ID,NAME,managerId
101,John,<null>,102,Dan,101
101,John,<null>,103,James,101
101,John,<null>,104,Amy,101
101,John,<null>,105,Anne,101
101,John,<null>,106,Ron,101
102,Dan,101,<null>,<null>,<null>
103,James,101,<null>,<null>,<null>
104,Amy,101,<null>,<null>,<null>
105,Anne,101,<null>,<null>,<null>
106,Ron,101,<null>,<null>,<null>


In [33]:
#!sql-sqlstudy
-- agora vamos incrementar a tabela com mais linhas e ver o novo resultado
INSERT INTO dbo.Employee (id, name, department, managerId) VALUES
(111, 'John',  'A', NULL),
(112, 'Dan',   'A', 111),
(113, 'James', 'A', 111),
(114, 'Amy',   'A', 111),
(115, 'Anne',  'A', 111),
(116, 'Ron',   'B', 111);


(6 rows affected)

In [37]:
#!sql-sqlstudy
-- VISAO GERAL, COM LEFT JOIN
SELECT 
    MANAG.ID AS MANAGER_ID, MANAG.NAME AS MANGER_NAME, MANAG.managerId AS MANAGER_MANAGER_ID,
    EMP.ID, EMP.NAME, EMP.managerId
FROM dbo.Employee MANAG
left JOIN dbo.Employee EMP ON MANAG.id = EMP.managerId
ORDER BY MANAG.ID, MANAG.NAME

-- RESOLVENDO COM AGREGACAO SIMPLES
SELECT
    MANAG.NAME AS MANAGE_NAME,
    COUNT(*) AS #MANAGES
FROM dbo.Employee MANAG
INNER JOIN dbo.Employee EMP ON MANAG.id = EMP.managerId
GROUP BY MANAG.ID, MANAG.NAME
HAVING COUNT(*) > 4

(20 rows affected)

(2 rows affected)

MANAGER_ID,MANGER_NAME,MANAGER_MANAGER_ID,ID,NAME,managerId
101,John,<null>,102,Dan,101
101,John,<null>,103,James,101
101,John,<null>,104,Amy,101
101,John,<null>,105,Anne,101
101,John,<null>,106,Ron,101
102,Dan,101,<null>,<null>,<null>
103,James,101,<null>,<null>,<null>
104,Amy,101,<null>,<null>,<null>
105,Anne,101,<null>,<null>,<null>
106,Ron,101,<null>,<null>,<null>


MANAGE_NAME,#MANAGES
John,5
John,5


In [39]:
#!sql-sqlstudy
-- agora vamos incrementar a tabela com mais linhas e ver o novo resultado
ALTER TABLE dbo.Employee
ALTER COLUMN name VARCHAR(100) NULL;
GO

INSERT INTO dbo.Employee (id, name, department, managerId) VALUES
(201, NULL, 'A', NULL),
(202, NULL, 'A', 201),
(203, NULL, 'A', 201),
(204, NULL, 'A', 201),
(205, NULL, 'A', 201),
(206, NULL, 'B', 201);



Commands completed successfully.

(6 rows affected)

In [41]:
#!sql-sqlstudy
-- VISAO GERAL, COM LEFT JOIN
SELECT 
    MANAG.ID AS MANAGER_ID, MANAG.NAME AS MANGER_NAME, MANAG.managerId AS MANAGER_MANAGER_ID,
    EMP.ID, EMP.NAME, EMP.managerId
FROM dbo.Employee MANAG
left JOIN dbo.Employee EMP ON MANAG.id = EMP.managerId
ORDER BY MANAG.ID, MANAG.NAME

-- VISAO GERAL, COM INNER JOIN
SELECT 
    MANAG.ID AS MANAGER_ID, MANAG.NAME AS MANGER_NAME, MANAG.managerId AS MANAGER_MANAGER_ID,
    EMP.ID, EMP.NAME, EMP.managerId
FROM dbo.Employee MANAG
INNER JOIN dbo.Employee EMP ON MANAG.id = EMP.managerId
ORDER BY MANAG.ID, MANAG.NAME

-- RESOLVENDO COM AGREGACAO SIMPLES
SELECT
    MANAG.NAME AS MANAGE_NAME,
    COUNT(*) AS #MANAGES
FROM dbo.Employee MANAG
INNER JOIN dbo.Employee EMP ON MANAG.id = EMP.managerId
GROUP BY MANAG.ID, MANAG.NAME
HAVING COUNT(*) > 4

(30 rows affected)

(15 rows affected)

(3 rows affected)

MANAGER_ID,MANGER_NAME,MANAGER_MANAGER_ID,ID,NAME,managerId
101,John,<null>,102,Dan,101
101,John,<null>,103,James,101
101,John,<null>,104,Amy,101
101,John,<null>,105,Anne,101
101,John,<null>,106,Ron,101
102,Dan,101,<null>,<null>,<null>
103,James,101,<null>,<null>,<null>
104,Amy,101,<null>,<null>,<null>
105,Anne,101,<null>,<null>,<null>
106,Ron,101,<null>,<null>,<null>


MANAGER_ID,MANGER_NAME,MANAGER_MANAGER_ID,ID,NAME,managerId
101,John,<null>,102,Dan,101
101,John,<null>,103,James,101
101,John,<null>,104,Amy,101
101,John,<null>,105,Anne,101
101,John,<null>,106,Ron,101
111,John,<null>,112,Dan,111
111,John,<null>,113,James,111
111,John,<null>,114,Amy,111
111,John,<null>,115,Anne,111
111,John,<null>,116,Ron,111


MANAGE_NAME,#MANAGES
John,5
John,5
<null>,5
